# Fase de post-exploitation en Windows (tras ganar acceso al sistema)

En la fase de post-explotación se mostrarán algunas técnicas que podemos aplicar tras haber comprometido un activo.

## NOTA

Antes de comenzar, por favor, aplicar la configuración de pivoting que se describe en el capítulo de las máquinas virtuales

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.

1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#privilegios">Escalación de privilegios</a>
  * Usando el fichero fodhelper.exe para escalar privilegios paso a paso
  * Usando el fichero fodhelper.exe para escalar privilegios desde el módulo bypassuac_fodhelper de empire
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#ejecutar_apps">Ejecutar programas</a>
  * Ejecutando comandos tras ganar acceso desde empire
  * Transferiendo la sesión de empire a metasploit
  * Ejecutando comandos desde metasploit
  * Ganando persistencia con el script *persistence* de metasploit
  * Pivoting con metasploit
  * Pivoting con proxychains levantando un servidor proxy con metasploit
<br><br>
3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#stego">Esteganografía</a>
  * Esteganografía con steghide
  * Esteganografía usando servicios online
<br><br>

### Referencias usadas

1. Páginas web
  * [Empire](https://www.powershellempire.com/)
  * [Manisfest files](https://msdn.microsoft.com/en-us/library/windows/desktop/aa375365%28v=vs.85%29.aspx)
  * [Fileless UAC bypass Fodhelper.exe](https://winscripting.blog/2017/05/12/first-entry-welcome-and-uac-bypass/)

<a name="privilegios"></a>

### Escalación de privilegios

Aparte de los métodos incluidos en el temario y las prácticas, vamos a ver cómo escalar privilegios usando ficheros ejecutables que elevan privilegios sin consultar al usuario:

####  Usando el fichero fodhelper.exe para escalar privilegios paso a paso

En la mayoría de sistemas operativos hay programas que deben ser ejecutados por usuarios sin privilegios pero que, debido a las acciones que realizan, necesitan permisos de administrador. Muchos de estos programas funcionan del siguiente modo:

  1. Son ejecutados por usuarios sin privilegios en el sistema operativo
  2. El programa eleva privilegios para poder realizar su función
  3. Tras realizar la función que requería permisos de administrador, el programa finaliza su ejecución
  
El programa *fodhelper.exe* es uno de ellos:

<img src="img/fodhelper_1.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a entender de manera informal cómo funciona este fichero y después que entendamos cómo funciona, lo vamos a usar para escalar privilegios. Para estudiar el comportamiento de los procesos (programas en ejecución), muchas veces necesitamos usar programas que necesitan privilegios de administrador luego, normalmente, se estudia el comportamiento del programa en un ordenador aparte y luego se aplican los conocimientos en equipo objetivo. Comenzamos descargando el programa *sigcheck* desde [windows sysinternals](https://docs.microsoft.com/en-us/sysinternals/downloads/sigcheck). Desde la web hacemos click en *Download Sigcheck*:

<img src="img/fodhelper_2.png" style="float:left">
<div style="clear:both"></div><br>

Descomprimimos el archivo, y accedemos a la carpeta donde lo hemos descomprimido desde la línea de comandos:

<img src="img/fodhelper_3.png" style="float:left">
<div style="clear:both"></div><br>

Lo ejecutamos pasandole como argumento el archivo del que queremos obtener información. En este caso queremos obtener información del archivo *fodhelper.exe* que se encuentra en la carpeta *System32* dentro de la carpeta *Windows*. Para ello, ejecutamos el comando *sigcheck* seguido de la ruta donde se encuentra el fichero:

*sigcheck %windir%\system32\fodhelper.exe*

<img src="img/fodhelper_4.png" style="float:left">
<div style="clear:both"></div><br>

La primera vez que ejecutemos el programa, nos solicitará que aceptemos los términos y condiciones. Si estamos de acuerdo, pulsamos *OK* para poder ejecutar el programa y analizar el fichero:

<img src="img/fodhelper_5.png" style="float:left">
<div style="clear:both"></div><br>

Tras ejecutarlo sin haberle pasado opciones, nos muestra información básica sobre el fichero:

<img src="img/fodhelper_6.png" style="float:left">
<div style="clear:both"></div><br>

Entre otros vemos que se trata de un archivo firmado digitalmente por Microsoft y la fecha de la firma. Vamos a ver el archivo [manisfest](https://msdn.microsoft.com/en-us/library/windows/desktop/aa375365%28v=vs.85%29.aspx) que acompaña a este binario. Grosso modo, los archivos manisfest son ficheros en formato *xml* que nos pueden aportar información muy útil sobre el ejecutable como, por ejemplom si puede autoelevar privilegios. Para ver el archivo *manifest* asociado a este binario, usamos la opción *-m* del programa *sigcheck*:

*sigcheck -m %windir%\system32\fodhelper.exe*

<img src="img/fodhelper_7.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que el archivo requiere permisos de administrador y que la propiedad *autoElevate* está activada. Que esta propiedad esté activada significa que el binario tiene permisos para elevar privilegios sin necesidad de preguntar al usuario. Normalmente, gracias al mecanismo [UAC](https://es.wikipedia.org/wiki/Control_de_cuentas_de_usuario) del sistema operativo windows, los programas que quieren elevar privilegios, preguntan primero al usuario para que lo autorice. Sin embargo si la propiedad *autoElevate* está activada, no preguntan al usuario y directamente elevan privilegios para realizar su tarea. Es decir, se saltan (bypass) el mecanismo de seguridad UAC del sistema operativo windows. 

Ya que sabemos que el fichero *fodhelper.exe* eleva privilegios en el sistema operativo sin consultar al usuario, vamos a observar qué acciones realiza el programa *fodhelper.exe* cuando se ejecuta. Para ello, vamos a utilizar otro programa de *windows sysinternals* llamado [Process Monitor](https://docs.microsoft.com/en-us/sysinternals/downloads/procmon) (procmon). Lo descargamos de la web oficial:

<img src="img/fodhelper_8.png" style="float:left">
<div style="clear:both"></div><br>

Process Monitor nos permite visualizar acciones que realiza un proceso (programa en ejecución) como ver a qué archivos accede o ver qué claves del registro de windows modifica. Una vez descomprimido, hacemos doble click para ejecutarlo:

<img src="img/fodhelper_9.png" style="float:left">
<div style="clear:both"></div><br>

Si estamos de acuerdo, aceptamos los términos y condiciones pulsando el botón *Agree*:

<img src="img/fodhelper_10.png" style="float:left">
<div style="clear:both"></div><br>

Le damos permiso para elevar privilegios:

<img src="img/fodhelper_11.png" style="float:left">
<div style="clear:both"></div><br>

Al abrirse, podremos ver una lista de los procesos y cada una de las acciones (Operations) que realizan, como por ejemplo, acceder o modificar claves del registro de windows, leer ficheros o crear [threads](https://es.wikipedia.org/wiki/Hilo_%28informática%29):

<img src="img/fodhelper_12.png" style="float:left">
<div style="clear:both"></div><br>

Una vez abierto, ejecutamos *fodhelper.exe* haciendo doble click en el archivo:

<img src="img/fodhelper_13.png" style="float:left">
<div style="clear:both"></div><br>

para que *Process Monitor* pueda monitorizar la actividad del proceso. Como el programa *fodhelper.exe* nos ayuda a gestionar características opcionales del sistema operativo, cuando hagamos doble click sobre el archivo nos aparecerá la siguiente pantalla que, simplemente, podemos ignorar:

<img src="img/fodhelper_14.png" style="float:left">
<div style="clear:both"></div><br>

Debido a que hay demasiada información, vamos a filtrar por acciones que este realizando el proceso *fodhelper*. Para ello, seleccionamos la opción *Filter...* del menú desplegable *Filter*:

<img src="img/fodhelper_15.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos *Process Name* en el primer desplegable:

<img src="img/fodhelper_16.png" style="float:left">
<div style="clear:both"></div><br>

Tecleamos *fodhelper.exe* en el cuadro de texto y pulsamos el botón *OK*:

<img src="img/fodhelper_17.png" style="float:left">
<div style="clear:both"></div><br>

A continuación hacemos click en *Yes*:

<img src="img/fodhelper_18.png" style="float:left">
<div style="clear:both"></div><br>

para filtrar la información:

<img src="img/fodhelper_19.png" style="float:left">
<div style="clear:both"></div><br>

Una vez aplicado el filtro, nos mostrará únicamente las acciones (Operations) realizadas por el proceso *fodhelper.exe*:

<img src="img/fodhelper_20.png" style="float:left">
<div style="clear:both"></div><br>

Aún así vemos que son muchísimas las acciones que realiza el proceso. Estudiar todo un proceso, se sale del objetivo de este manual pero, la idea es ir viendo las acciones que hace el proceso y ver si podemos sacar partido de alguna de ellas. En este caso, como ya está documentado, vamos ver cómo podemos aprovecharnos de ciertas acciones que realiza el proceso *fodhelper.exe*. Para ello, vamos a fijarnos en la operaciones que realiza en el registro de windows. Sobre todo aquellas que realiza sobre la claves en *HKCU (HKEY_CURRENT_USER)* ya que podemos modificar estas claves (valores) del registro sin tener permisos de administrador. En este caso, el investigador se fijo en los accesos a claves del registro de windows que no existen. Específicamente encontró esta clave:

*HKCU\Software\Classes\ms-settings\shell\Open\Command*

<img src="img/fodhelper_21.png" style="float:left">
<div style="clear:both"></div><br>

que si abrimos el registro de windows:

<img src="img/fodhelper_22.png" style="float:left">
<div style="clear:both"></div><br>

(aceptamos que se abra con permisos de administrador):

<img src="img/fodhelper_23.png" style="float:left">
<div style="clear:both"></div><br>

podremos comprobar que no existe:

<img src="img/fodhelper_24.png" style="float:left">
<div style="clear:both"></div><br>

Si creamos esta clave en el registro. Por ejemplo, podemos crearla desde la línea de comandos:

<img src="img/fodhelper_25.png" style="float:left">
<div style="clear:both"></div><br>

Usando el comando *reg* con la opción *add*:

<img src="img/fodhelper_26.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos *F5* para refrescar el registro y podremos ver que la clave se ha añadido al registro:

<img src="img/fodhelper_26_1.png" style="float:left">
<div style="clear:both"></div><br>

Abrimos nuevamente *Process Monitor* y ejecutamos *fodhelper.exe* de nuevo, veremos que ahora, tras encontrar la clave que hemos creado, intenta encontrar el valor *DelegateExecute* en la clave:

*HKCU\Software\Classes\ms-settings\shell\open\command\DelegateExecute*

pero no la encuentra (NAME NOT FOUND):

<img src="img/fodhelper_27.png" style="float:left">
<div style="clear:both"></div><br>

repetimos el mismo proceso, la creamos el valor desde la línea de comandos pasandole la opción */v* para indicarle el nombre del valor:

<img src="img/fodhelper_28.png" style="float:left">
<div style="clear:both"></div><br>

Comprobamos nuevamente que se ha creado en el registro:

<img src="img/fodhelper_29.png" style="float:left">
<div style="clear:both"></div><br>

y volvemos a ejecutar *Process Monitor* y después *fodhelper.exe*. Ahora vemos que, tras encontrar el valor *DelegateExecute* intenta leer el valor por defecto (default) en la clave:

*HKCU\Software\Classes\ms-settings\shell\open\command*

<img src="img/fodhelper_30.png" style="float:left">
<div style="clear:both"></div><br>

Luego, si modificamos este valor por defecto (default), nos ejecutará cualquier comando que le indiquemos. Por ejemplo, vamos a indicar que ejecute *powershell*. Para ello, nuevamente desde la línea de comandos, ejecutamos el comando *reg* pasandole la opción */d* para indicarle los datos del valor (Al ser el valor por defecto no necesitamos usar la opción */v*):

<img src="img/fodhelper_31.png" style="float:left">
<div style="clear:both"></div><br>

Comprobamos que el valor se ha cambiado en el registro:

<img src="img/fodhelper_32.png" style="float:left">
<div style="clear:both"></div><br>

Y ahora si ejecutamos *fodhelper.exe* haciendo doble click en el archivo:

<img src="img/fodhelper_34.png" style="float:left">
<div style="clear:both"></div><br>

se abrirá una sesión de *powershell* con privilegios de administrador:

<img src="img/fodhelper_33.png" style="float:left">
<div style="clear:both"></div><br>

Como podeis observar, para escalar privilegios, sólo sería necesario ejecutar los comandos *reg* para añadir la clave del registro, añadir el valor *DelegateExecute* en la clave y cambiar los datos por defecto en el valor *Default*. Estos cambios los podemos realizar como usuarios porque la clave que modificamos es *HKCU*. Una vez comprendido el proceso de forma manual, vamos a ver como hacerlo de forma automática con el framework *empire*.

#### Usando el fichero fodhelper.exe para escalar privilegios desde el módulo bypassuac_fodhelper de empire

Ahora que hemos visto como hacer la escalación de forma manual, veremos que sencillo es hacerlo usando empire. Para ello vamos a conectar la máquina windows 10 comprometida con empire. La instalación del framework empire y como crear una conexión simple lo vimos en el capítulo anterior. Para ello, desde Kali Linux, ejecutamos el framework empire:

<img src="img/empire_bypassuac_1.png" style="float:left">
<div style="clear:both"></div><br>

Accedemos a las opciones para crear un *listener* (servicio a la escucha) con el comando *listeners*:

<img src="img/empire_bypassuac_2.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a crear un *listener* (servicio) http con el comando *uselistener*.

*uselistener http*

<img src="img/empire_bypassuac_3.png" style="float:left">
<div style="clear:both"></div><br>

Le cambiamos el nombre a *http-no-priv*:

<img src="img/empire_bypassuac_4.png" style="float:left">
<div style="clear:both"></div><br>

Dejamos las opciones por defecto y lo ejecutamos con el comando *execute*:

<img src="img/empire_bypassuac_5.png" style="float:left">
<div style="clear:both"></div><br>

Volvemos al menú *listeners* usando el comando *listeners* ya que así podremos ver que el listener que hemos creado funciona correctamente:

<img src="img/empire_bypassuac_6.png" style="float:left">
<div style="clear:both"></div><br>

y creamos el *launcher* para nuestro *listener* (servicio) usando el comando:

*launcher powershell http-no-priv*

<img src="img/empire_bypassuac_7.png" style="float:left">
<div style="clear:both"></div><br>

A continuación copiamos el script de powershell:

<img src="img/empire_bypassuac_8.png" style="float:left">
<div style="clear:both"></div><br>

y nos vamos a la máquina de windows 10. Abrimos el intérprete de comandos en windows 10. En este caso lo abrimos como usuarios, tecleamos *cmd* y hacemos click:

<img src="img/empire_bypassuac_9.png" style="float:left">
<div style="clear:both"></div><br>

Pegamos el script:

<img src="img/empire_bypassuac_10.png" style="float:left">
<div style="clear:both"></div><br>

y lo ejecutamos presionado intro. Tras ejecutarlo, veremos que la máquina con windows 10 ha sido comprometida porque se nos ha creado un *agent* (máquina comprometida) en Empire:

<img src="img/empire_bypassuac_11.png" style="float:left">
<div style="clear:both"></div><br>

Presionamos intro para que vuelva a aparecer el prompt y tecleamos *agents* para ver la máquina que hemos comprometido y acceder al menú de los *agents* 

<img src="img/empire_bypassuac_12.png" style="float:left">
<div style="clear:both"></div><br>

Cambiamos el nombre al *agent* utilizando el comando *rename* por uno que nos resulte más fácil de recordar:

<img src="img/empire_bypassuac_13.png" style="float:left">
<div style="clear:both"></div><br>

Ya podemos interactuar con el *agent* (maquina comprometida) utilizando el comando *interact* seguido del nombre del agente:

<img src="img/empire_bypassuac_14.png" style="float:left">
<div style="clear:both"></div><br>

Como no tenemos privilegios en la máquina comprometida, si intentamos ejecutar algún comando que requiera permisos de administrador como *mimikatz*, obtendremos el siguiente error:

<img src="img/empire_bypassuac_15.png" style="float:left">
<div style="clear:both"></div><br>

Para escalar privilegios podemos usar alguno de los *modules* de *privesc/bypassuac*. Pero antes de poder ejecutarlos, debemos crear un nuevo *listener* (servicio) a la escucha para recibir la conexión de la sesión con privilegios. Para ello, vamos al menú listeners:

<img src="img/empire_bypassuac_16.png" style="float:left">
<div style="clear:both"></div><br>

Creamos un nuevo *listener* (servicio) http:

<img src="img/empire_bypassuac_17.png" style="float:left">
<div style="clear:both"></div><br>

le damos un nombre:

<img src="img/empire_bypassuac_18.png" style="float:left">
<div style="clear:both"></div><br>

le cambiamos el puerto porque ya tenemos un servicio (*listener*) en el puerto 80. Por ejemplo, podemos usar el puerto 81:

<img src="img/empire_bypassuac_19.png" style="float:left">
<div style="clear:both"></div><br>

Nos aseguramos que el host sea correcto fijandolo manualmente:

<img src="img/empire_bypassuac_19_1.png" style="float:left">
<div style="clear:both"></div><br>

lo ejecutamos:

<img src="img/empire_bypassuac_20.png" style="float:left">
<div style="clear:both"></div><br>

y comprobamos que nuestro listener está activo:

<img src="img/empire_bypassuac_20_1.png" style="float:left">
<div style="clear:both"></div><br>

Ahora si que podemos volver al menú de *agents* (máquinas comprometidas):

<img src="img/empire_bypassuac_21.png" style="float:left">
<div style="clear:both"></div><br>

e interactuar con nuestra máquina comprometida:

<img src="img/empire_bypassuac_22.png" style="float:left">
<div style="clear:both"></div><br>

Para escalar privilegios, vamos a aprovecharnos del fichero [fodhelper.exe](https://winscripting.blog/2017/05/12/first-entry-welcome-and-uac-bypass/) usando el modulo *privesc/bypassuac_fodhelper*. Para ello usamos el comando:

*usemodule privesc/bypassuac_fodhelper*

<img src="img/empire_bypassuac_23.png" style="float:left">
<div style="clear:both"></div><br>

ejecutamos el comando *options* para ver las opciones que son necesarias:

<img src="img/empire_bypassuac_24.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, sólo necesitamos indicar el *listener* (servicio) a donde queremos que se conecte la sesión con privilegios. Para indicarle el *listener* ejecutamos:

*set Listener http-con-priv*

<img src="img/empire_bypassuac_25.png" style="float:left">
<div style="clear:both"></div><br>

y lo ejecutamos con el comando *execute*:

<img src="img/empire_bypassuac_26.png" style="float:left">
<div style="clear:both"></div><br>

Aunque no sea *opsec safe*, aceptamos ejecutarlo y se creará una sesión con privilegios:

<img src="img/empire_bypassuac_27.png" style="float:left">
<div style="clear:both"></div><br>

Para ver el nuevo *agent* con privilegios, tecleamos el comando *agents*:

<img src="img/empire_bypassuac_28.png" style="float:left">
<div style="clear:both"></div><br>

Le damos al *agent* (máquina comprometida) un nombre más sencillo de recordar usando el comando *rename*:

<img src="img/empire_bypassuac_29.png" style="float:left">
<div style="clear:both"></div><br>

e interactuamos con la nueva sesión con privilegios usando el comando *interact*:

<img src="img/empire_bypassuac_30.png" style="float:left">
<div style="clear:both"></div><br>

podemos comprobar que ahora si tenemos permisos de administrador ejecutando el comando *mimikatz*:

<img src="img/empire_bypassuac_31.png" style="float:left">
<div style="clear:both"></div><br>

ahora que tenemos privilegios de administrador, podemos seguir escalando para tener el máximo nivel de privilegios. Los de la cuenta *AUTHORITY/SYSTEM*. Para ello usamos el modulo *privesc/getsystem*:

*usemodule privesc/getsystem*

<img src="img/empire_bypassuac_32.png" style="float:left">
<div style="clear:both"></div><br>

Si ejecutamos *options*:

<img src="img/empire_bypassuac_33.png" style="float:left">
<div style="clear:both"></div><br>

veremos que no nos hace falta completar ninguna opción para ejecutarlo. Por lo que simplemente, lo ejecutamos:

<img src="img/empire_bypassuac_34.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que tengamos el máximo nivel de privilegios en el *agent* (máquina comprometida). Podemos usar el comando *back* para volver a interactuar con el *agent* (sesión en la máquina comprometida):

<img src="img/empire_bypassuac_35.png" style="float:left">
<div style="clear:both"></div><br>

y confirmar el nivel de privilegios tenemos consultando el usuario que tenemos en la máquina con el comando *whoami*:

<img src="img/empire_bypassuac_36.png" style="float:left">
<div style="clear:both"></div><br>

<a name="ejecutar_apps"></a>

### Ejecutar programas

#### Ejecutando comandos tras ganar acceso desde empire

Tras escalar privilegios, vamos a ver algunos comandos que podemos ejecutar para o bien aportar evidencias del acceso a la máquina o bien seguir realizando acciones en la fase de post-explotation. Uno de los comandos que podemos utilizar para tener una evidencia que hemos estado en la máquina es hacer una captura de pantalla. Para ello, podemos usar el comando *sc*:

<img src="img/empire_agent_commands_1.png" style="float:left">
<div style="clear:both"></div><br>

Veremos que nos guarda en el directorio *downloads* del framework empire. Para visualizarla accedemos al directorio donde tengamos descargado empire:

<img src="img/empire_agent_commands_2.png" style="float:left">
<div style="clear:both"></div><br>

y desde allí, ejecutamos el comando *display* pasandole como argumento la ruta donde está la imagen:

<img src="img/empire_agent_commands_3.png" style="float:left">
<div style="clear:both"></div><br>

para visualizarla:

<img src="img/empire_agent_commands_4.png" style="float:left">
<div style="clear:both"></div><br>

También podemos obtener información del sistema operativo con el comando *sysinfo*:

<img src="img/empire_agent_commands_5.png" style="float:left">
<div style="clear:both"></div><br>

#### Transferiendo la sesión de empire a metasploit

[Metasploit](https://www.metasploit.com/) es un framework que nos puede ayudar mucho en la fase de post-explotación. Podemos crear una sesion en metasploit desde empire. La forma en que lo vamos a hacer es descargar un payload y ejecutarlo. Como windows defender no nos va a permitir realizar esta acción, primero debemos desactivarlo. Para ello, utilizamos el comando *shell* para ejecutar comandos en el *agent* (máquina comprometida) y utilizamos el cmdlet (comando) *Set-MpPreference* de powershell pasandole como argumento *-DisableRealtimeMonitoring*:

*shell powershell Set-MpPreference -DisableRealtimeMonitoring 1*

<img src="img/empire_to_metasploit_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez deshabilitado windows defender, hacemos click en el icono metasploit:

<img src="img/empire_to_metasploit_2.png" style="float:left">
<div style="clear:both"></div><br>

para iniciarlo:

<img src="img/empire_to_metasploit_3.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a configurar un script para hacer el delivery de meterpreter y crear un servicio a la escucha para recibir la conexión:

*use exploit/multi/script/web_delivery*

<img src="img/empire_to_metasploit_4.png" style="float:left">
<div style="clear:both"></div><br>

Miramos las opciones que debemos configurar con el comando *options*:

<img src="img/empire_to_metasploit_5.png" style="float:left">
<div style="clear:both"></div><br>

Debemos configurar:

  * **payload** Para indicar el payload a utilizar. En este caso usaremos *windows/meterpreter/reverse_tcp*
  * **SRVHOST** Para indicar el host desde la que se lanzará el payload. En este caso la IP de Kali Linux
  * **SRVPORT** Para indicar el puerto en el que se levantará el servicio web desde el que se servirá el payload
  * **URIPATH** La URI desde la que se servirá el payload
  * **LHOST** Para indicar el host al que debe conectarse el payload una vez se ejecute en windows 10
  * **LPORT** Para indicar el puerto al que debe conectarse el payload una vez se ejecute en windows 10
  * **target** Para indicarle en que lenguaje de scripting queremos el payload. En este caso al ser windows 10, usaremos powershell
  
Comenzamos indicando el payload a utilizar *windows/meterpreter/reverse_tcp*:

<img src="img/empire_to_metasploit_6.png" style="float:left">
<div style="clear:both"></div><br>

Configuramos el host desde la que se lanzará el payload:

<img src="img/empire_to_metasploit_7.png" style="float:left">
<div style="clear:both"></div><br>

Indicamos el puerto en el que se levantará el servicio web desde el que se servirá el payload:

<img src="img/empire_to_metasploit_8.png" style="float:left">
<div style="clear:both"></div><br>

Continuamos con la URI desde la que se servirá el payload:

<img src="img/empire_to_metasploit_9.png" style="float:left">
<div style="clear:both"></div><br>

Indicamos host al que debe conectarse el payload una vez se ejecute en windows 10:

<img src="img/empire_to_metasploit_10.png" style="float:left">
<div style="clear:both"></div><br>

y el puerto:

<img src="img/empire_to_metasploit_11.png" style="float:left">
<div style="clear:both"></div><br>

finalmente elegimos powershell como lenguaje de scripting para el payload:

<img src="img/empire_to_metasploit_12.png" style="float:left">
<div style="clear:both"></div><br>

Lo ejecutamos con el comando *run*:

<img src="img/empire_to_metasploit_13.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos la URL y después hacemos click con el botón derecho del ratón sobre ella desplegar el menú. Seleccionamos *Copy* para copiarla:

<img src="img/empire_to_metasploit_14.png" style="float:left">
<div style="clear:both"></div><br>

Esta es la URL desde donde se servirá el payload. Volvemos a interactuar con el *agent* y usaremos el module *code_execution/invoke_metasploitpayload*:

<img src="img/empire_to_metasploit_15.png" style="float:left">
<div style="clear:both"></div><br>

Ejecutamos el comando *options* para ver qué datos necesitamos aportar al módulo:

<img src="img/empire_to_metasploit_16.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que sólo es necesario aportar la URL que ha generado metasploit. Le indicamos la URL:

<img src="img/empire_to_metasploit_17.png" style="float:left">
<div style="clear:both"></div><br>

Lo ejecutamos con el comando *execute*:

<img src="img/empire_to_metasploit_18.png" style="float:left">
<div style="clear:both"></div><br>

Veremos que se nos genera una nueva sesión en metasploit:

<img src="img/empire_to_metasploit_19.png" style="float:left">
<div style="clear:both"></div><br>

Aunque lo ideal es que la sesión se crease con permisos *AUTHORITY/SYSTEM*, la sesión se crea con permisos de *Administrador* ya que escalamos privilegios. Pulsamos intro y ejecutamos el comando *sessions* pasandole la opción *-l* para ver las sesiones:

<img src="img/empire_to_metasploit_20.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que el *Id* de la sesión es 1. Usaremos esta sesión para interactuar con la máquina. Para ello, volvemos a usar el comando *sessions* pasandole como argumento el *Id* de la sesión:

<img src="img/empire_to_metasploit_21.png" style="float:left">
<div style="clear:both"></div><br>

#### Ejecutando comandos desde metasploit

Como la sesión se crea con permisos de *administrator*, volvemos a escalar privilegios para ser *AUTHORITY/SYSTEM* usando el comando *getsystem* de meterpreter:

<img src="img/metasploit_comandos_1.png" style="float:left">
<div style="clear:both"></div><br>

Al igual que ocurría con empire, podemos usar el comando *sysinfo* para obtener información del sistema:

<img src="img/metasploit_comandos_2.png" style="float:left">
<div style="clear:both"></div><br>

Podemos usar *pwd* para saber el directorio en el que nos encontramos:

<img src="img/metasploit_comandos_3.png" style="float:left">
<div style="clear:both"></div><br>

También podemos hacer capturas de pantalla con el comando *screenshot*:

<img src="img/metasploit_comandos_4.png" style="float:left">
<div style="clear:both"></div><br>

Podemos usar el comando *display* desde el interprete de comandos:

<img src="img/metasploit_comandos_5.png" style="float:left">
<div style="clear:both"></div><br>

para ver la imagen:

<img src="img/metasploit_comandos_6.png" style="float:left">
<div style="clear:both"></div><br>


#### Ganando persistencia con el script persistence de metasploit

Podemos ejecutar el script *persistence* de metasploit para obtener persistencia en la máquina comprometida. Para ver todas las opciones disponibles del script usamos el comando:

*run persistence -h*

<img src="img/metasploit_persistence_1.png" style="float:left">
<div style="clear:both"></div><br>

En este caso le vamos a pasar las siguientes opciones:

  * **-r {host}** para indicarle el host al que queremos que se conecte (nuestra máquina de Kali Linux)
  * **-p {puerto}** para indicarle a qué puerto queremos que inicie la conexión
  * **-U** para indicarle que queremos que se conecte cuando el usuario inicie sesión

<img src="img/metasploit_persistence_2.png" style="float:left">
<div style="clear:both"></div><br>

#### Pivoting con metasploit

Una vez comprometemos un equipo, podemos obtener acceso a otras redes internas que acceso ese equipo. Ejecutamos el comando *ipconfig* para ver las redes disponibles:

<img src="img/metasploit_pivoting_1.png" style="float:left">
<div style="clear:both"></div><br>

Entre los resultados, vemos la red a la que hemos ganado acceso:

<img src="img/metasploit_pivoting_2.png" style="float:left">
<div style="clear:both"></div><br>

y vemos otra red:

<img src="img/metasploit_pivoting_3.png" style="float:left">
<div style="clear:both"></div><br>

Vamos a acceder a esta red para descubrir nuevos equipos que esten accesibles desde esta red. Para ello, ejecutamos el script *autoroute* de metasploit con la opción *-s* pasandole como argumento la red a la que queremos pivotar:

*run autoroute -s 192.168.119.0/24*

<img src="img/metasploit_pivoting_4.png" style="float:left">
<div style="clear:both"></div><br>

Comprobamos que se ha añadido correctamente a la tabla de enrutamiento usando el comando:

*run autoroute -p*

<img src="img/metasploit_pivoting_5.png" style="float:left">
<div style="clear:both"></div><br>

Es decir, ahora tenemos acceso a esta red usando la sesión 1 en metasploit. Vamos a ver un ejemplo. Ponemos la sesión en segundo plano usando el comando *background*:

<img src="img/metasploit_pivoting_6.png" style="float:left">
<div style="clear:both"></div><br>

Recordar que podemos ver las sesiones en segundo plano usando el comando *sessions* y pasandole la opción *-l*:

*sessions -l*

<img src="img/metasploit_pivoting_7.png" style="float:left">
<div style="clear:both"></div><br>

Por ejemplo, vamos a usar un scanner de metasploit para descubrir los servicios ftp y web que están activos en la nueva red. Para ello, podemos usar el scanner:

*use auxiliary/scanner/portscan/tcp*

<img src="img/metasploit_pivoting_8.png" style="float:left">
<div style="clear:both"></div><br>

Utilizamos el comando *show options* para ver qué opciones tenemos que pasarle al scanner:

<img src="img/metasploit_pivoting_9.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, le vamos a indicar la red que queremos escanear *192.168.119.0/24*:

<img src="img/metasploit_pivoting_10.png" style="float:left">
<div style="clear:both"></div><br>

y los puertos que queremos. en este caso el 21 y el 80:

<img src="img/metasploit_pivoting_11.png" style="float:left">
<div style="clear:both"></div><br>

finalmente, lo ejecutamos para escanear la red en busca de servicios ftp y web:

<img src="img/metasploit_pivoting_12.png" style="float:left">
<div style="clear:both"></div><br>

El escáner es lento pero, a medida que vaya encontrando equipos en esa red con servicios en los puertos 21 y 80, nos los irá mostrando:

<img src="img/metasploit_pivoting_13.png" style="float:left">
<div style="clear:both"></div><br>

Una vez finalizado el escaneo, nos informará de todas servicios ftp y http que ha encontrado en la nueva red:

<img src="img/metasploit_pivoting_14.png" style="float:left">
<div style="clear:both"></div><br>

#### Pivoting con proxychains levantando un servidor proxy con metasploit

El problema de hacer pivoting del modo anterior es que sólo podemos usar los comandos de *meterpreter* para interactuar con las máquinas y servicios que encontremos en la nueva red. Para poder usar cualquier tipo de programa, como por ejemplo *nmap*, podemos usar metasploit para levantar un servidor proxy que nos lleve a esta red y usar el programa *proxychains* para configurarlo. Para levantar un servidor proxy con metasploit, usaremos el módulo *auxiliary/server/socks4a*:

*use auxiliary/server/socks4a*

<img src="img/metasploit_pivoting_15.png" style="float:left">
<div style="clear:both"></div><br>

Como siempre, utilizamos *show options* para ver las opciones que tenemos que completar. En este caso las dejamos por defecto:

<img src="img/metasploit_pivoting_16.png" style="float:left">
<div style="clear:both"></div><br>

y lo ejecutamos:

<img src="img/metasploit_pivoting_17.png" style="float:left">
<div style="clear:both"></div><br>

Una vez levantado el proxy, editamos el fichero de configuración de proxychains (*/etc/proxychains.conf*) para añadir el proxy. En este caso usamos el editor vim pra editar el fichero:

<img src="img/metasploit_pivoting_18.png" style="float:left">
<div style="clear:both"></div><br>

Comentamos la línea de *tor* y añadimos nuestro *proxy*:

<img src="img/metasploit_pivoting_19.png" style="float:left">
<div style="clear:both"></div><br>

Una vez añadido el proxy, ya podemos usar cualquier programa, como nmap, a través de proxychains para llegar a los equipos y servicios de la nueva red:

<img src="img/metasploit_pivoting_20.png" style="float:left">
<div style="clear:both"></div><br>

<a name="stego"></a>

### Esteganografía

La [esteganografía](https://es.wikipedia.org/wiki/Esteganografía) estudía métodos y técnicas para ocultar información o archivos, como imagenes o videos, en otros archivos. Por ejemplo, esconder mensajes en ficheros de audio o en imágenes.

#### Esteganografía con steghide

Steghide es un programa que podemos usar en Kali Linux para esconder un texto en una imagen. Para usarlo, primero tenemos que instalarlo:

<img src="img/steghide_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez instalado, descargamos buscamos una imagen donde esconder el texto. En este caso yo descargo una imagen pública y la llamo railway.jpeg:

*wget https://images.pexels.com/photos/213181/pexels-photo-213181.jpeg -O railway.jpeg*

<img src="img/steghide_2.png" style="float:left">
<div style="clear:both"></div><br>

Ahora creamos un fichero de texto con el mensaje a esconder. En este redirecciono la salida del comando *echo* a un fichero:

*echo Mensaje para ocultar en la imagen > oculto.txt*

<img src="img/steghide_3.png" style="float:left">
<div style="clear:both"></div><br>

Si ejecutamos ls, veremos los 2 ficheros:

<img src="img/steghide_4.png" style="float:left">
<div style="clear:both"></div><br>

Para ocultar el texto en la imagen, ejecutamos el comando *steghide* con las siguientes opciones:

  * **embed** => para indicar que queremos incluir un fichero dentro de otro
  * **-ef {archivoDeTextoAOcultar}** => para indicar el archivo de texto que queremos ocultar
  * **-cv {arhivoDondeSeOcultará}** => para indicar el fichero donde ocultaremos el texto
  
Como queremos ocultar el fichero *oculto.txt* en la imagen *railway.jpeg*:

*steghide embed -ef oculto.txt -cf railway.jpeg*

<img src="img/steghide_5.png" style="float:left">
<div style="clear:both"></div><br>

Tras ejecutar el comando, nos solicitará que metamos una contraseña y, tras introducirla, que la confirmemos:

<img src="img/steghide_6.png" style="float:left">
<div style="clear:both"></div><br>

Tras confirmarla, nos indicará que se ha ocultado con éxito:

<img src="img/steghide_7.png" style="float:left">
<div style="clear:both"></div><br>

Borramos el fichero *oculto.txt*:

<img src="img/steghide_8.png" style="float:left">
<div style="clear:both"></div><br>

Ahora sólo nos queda la imagen:

<img src="img/steghide_9.png" style="float:left">
<div style="clear:both"></div><br>

Si visualizamos la imagen:

<img src="img/steghide_10.png" style="float:left">
<div style="clear:both"></div><br>

no notaremos ningún cambio:

<img src="img/steghide_11.png" style="float:left">
<div style="clear:both"></div><br>

Para extraer el mensaje oculto de la imagen, ejecutamos el comando *steghide* con las siguientes opciones:

  * **extract** => para indicar que queremos extraer un fichero de la imagen
  * **-sf {archivoQueContieneElFicheroOcultado}** => para indicar el archivo de texto que contiene el fichero ocultado

*steghide extract -sf railway.jpeg*

<img src="img/steghide_12.png" style="float:left">
<div style="clear:both"></div><br>

Tras meter la contraseña, veremos que el fichero oculto se extrae de la imagen:

<img src="img/steghide_13.png" style="float:left">
<div style="clear:both"></div><br>

y lo podremos ver en el directorio:

<img src="img/steghide_14.png" style="float:left">
<div style="clear:both"></div><br>

#### Esteganografía usando servicios online

Aunque no es aconsejable usarlos, existen muchos servicios web que me van a permitir realizar esteganografía de un modo más sencillo. Vamos a ver como usar uno de ellos. Accedemos a [Mobile Fish](https://www.mobilefish.com/services/steganography/steganography.php) y desplazamos el cursor hasta encontrar la parte que nos permite esconder texto en una imagen:

<img src="img/online_stego_1.png" style="float:left">
<div style="clear:both"></div><br>

Pulsamos el botón *Seleccionar archivo* para elegir la imágen:

<img src="img/online_stego_2.png" style="float:left">
<div style="clear:both"></div><br>

Tras elegir la imagen, pulsamos el botón *Open*:

<img src="img/online_stego_3.png" style="float:left">
<div style="clear:both"></div><br>

Después tecleamos el texto que queramos ocultar y metemos el código de acceso que nos indica en el recuadro negro de letras blancas:

<img src="img/online_stego_4.png" style="float:left">
<div style="clear:both"></div><br>

Finalmente, pulsamos el enlace *Download file* para descargar el fichero:

<img src="img/online_stego_5.png" style="float:left">
<div style="clear:both"></div><br>

Veremos que se descarga el la imágen con el fichero oculto:

<img src="img/online_stego_6.png" style="float:left">
<div style="clear:both"></div><br>

Para extraer el mensaje, podemos utilizar nuevamente la web de [mobilefish](https://www.mobilefish.com/services/steganography/steganography.php).